In [1]:
import sys
sys.path.append("../")

In [2]:
# imports
from flow.solve_mix import *
# from flow.build_circuits import *  # Takes some time to import
from utils.sic_fiducial_state import *
from utils.inner_product import *
from utils.handy_states import *
from utils.inner_product import inner_products_table
from utils.multiple_wigner import plot_multiple_wigner
from utils.complex_heatmap import ComplexHeatmap
from scipy.spatial.distance import jensenshannon
c_heatmap = ComplexHeatmap()

In [3]:
# Input states
num_qubits = 2
num_states = 3
state_vec = sv_simple_2(0.2, 0.5, 0.7)["states"]

In [4]:
for state in state_vec:
    print(state)

Statevector([0.98058068+0.j, 0.        +0.j, 0.        +0.j,
             0.19611614+0.j],
            dims=(2, 2))
Statevector([0.        +0.j, 0.89442719+0.j, 0.        +0.j,
             0.4472136 +0.j],
            dims=(2, 2))
Statevector([0.        +0.j, 0.        +0.j, 0.81923192+0.j,
             0.57346234+0.j],
            dims=(2, 2))


In [5]:
inner_products_table([item.data for item in state_vec])
print()


Inner Product Table:
----------------------------------------------------------------------
|  States   |     State 0      |     State 1      |     State 2      |
--------------------------------------------------------------------
|  State 0  |      1.e+00      |    8.771e-02     |    1.125e-01     |
|  State 1  |    8.771e-02     |      1.e+00      |    2.565e-01     |
|  State 2  |    1.125e-01     |    2.565e-01     |      1.e+00      |
----------------------------------------------------------------------



In [6]:
dense_mat = [DensityMatrix(_) for _ in state_vec]

problem_asymm_states_1 = ProblemSpec(
    num_qubits=num_qubits,
    num_states=num_states,
    seed=33,
    case_id=f"q{num_qubits}_n{num_states}_symm",
    state_type="statevector",
)
problem_asymm_states_1.set_states(
    state_type="statevector",
    states=state_vec,
    overwrite=True,
)
print("Is linear indep", problem_asymm_states_1.is_lin_indep())
ideal_result = apply_Eldar(problem_spec=problem_asymm_states_1)

print(verify_povm_matrix(ideal_result["povm"]))
p_inc_ideal = ideal_result["p_inc"]
len(ideal_result["povm"])

Rank = 3
#states = 3
Is linear indep True
True


4

In [7]:
p_succ_ideal = sum(np.multiply(ideal_result['sol'], 1/3))
ideal_distrib = np.concatenate((np.multiply(ideal_result['sol'], 1/3), np.array([1. - p_succ_ideal])))
print(ideal_distrib)

[0.3205 0.2667 0.2237 0.1891]


In [8]:
noise_levels = [0.1 ** (6 - 0.5 * i) for i in range(11)]
disturbance_states = [
    DensityMatrix(
        ProblemSpec.depolarizing_noise_channel(num_qubits=num_qubits)
    )
    for _ in range(num_states)
]

In [9]:
def report_ideal(ideal_distrib):
    print("Ideal input states")
    print("POVM for ideal optimal UQSD")
    print(ideal_distrib)
    print()

def dist_1(m):
    correct_results = [m[i][i] for i in range(len(m))]
    p_succ_ni = sum(correct_results)
    return np.concatenate((correct_results, np.array([1. - p_succ_ni])))

def dist_2(m):
    measured_results = [sum([m[j][i] for j in range(len(m))]) for i in range(len(m))]
    p_succ_ni = sum(measured_results)
    return np.concatenate((measured_results, np.array([1. - p_succ_ni])))

def get_sqrt_dist(a, b):
    return np.sqrt(sum([abs(a[i] - b[i]) ** 2 for i in range(len(a))]))

def report_noisy_states_with_ideal_povm(noise_level, ideal_distrib):
    combined_states = [
        (1 - noise_level) * dense_mat[_]
        + noise_level * disturbance_states[_].data
        for _ in range(num_states)
    ]
    
    problem_asymm_states_1.set_states(
        state_type="densitymatrix",
        states=combined_states,
        overwrite=True,
    )
    # m is prob_matrix
    m = calculate_prob_matrix_simple(problem_asymm_states_1.prior_prob, ideal_result["povm"], combined_states)
    # for i in m:
    #     print(i)
    noisy_distrib_with_ideal_povm = dist_2(m)
    print(f"Noisy input states with depolarizing noise = {noise_level:3g}")
    print("POVM for ideal optimal UQSD")
    print(noisy_distrib_with_ideal_povm)
    np.set_printoptions(precision=4)
    print("alpha", np.array(calculate_errors(m)[0]))
    print("beta ", np.array(calculate_errors(m)[1]))
    js_dist = jensenshannon(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Jensen-Shannon dist {js_dist:3f}")
    sqrt_dist = get_sqrt_dist(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Root sum of squares {sqrt_dist:3f}")
    print()


In [10]:
def report_noisy_states_with_FRIO_povm(noise_level, ideal_distrib):
    combined_states = [
        (1 - noise_level) * dense_mat[_]
        + noise_level * disturbance_states[_].data
        for _ in range(num_states)
    ]
    
    problem_asymm_states_1.set_states(
        state_type="densitymatrix",
        states=combined_states,
        overwrite=True,
    )
    noisy_result = apply_Eldar_mix_primal(
        problem_spec=problem_asymm_states_1,
        beta=ideal_distrib[-1],
        is_cvxpy_verbose=False,
    )
    # m is prob_matrix
    m = calculate_prob_matrix_simple(
        prior_probs=problem_asymm_states_1.prior_prob,
        povm=noisy_result["PI_list"],
        states=combined_states,
        ## bitstring_to_target_state=result["bitstring_to_target_state"],
        ## strings_used=result["strings_used"],
    )
    # for i in m:
    #     print(i)
    noisy_distrib_with_ideal_povm = dist_2(m)
    print(f"Noisy input states with depolarizing noise = {noise_level:3g}")
    print("POVM for FRIO-optimal UQSD")
    print(noisy_distrib_with_ideal_povm)
    np.set_printoptions(precision=4)
    print("alpha", np.array(calculate_errors(m)[0]))
    print("beta ", np.array(calculate_errors(m)[1]))
    js_dist = jensenshannon(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Jensen-Shannon dist {js_dist:3f}")
    sqrt_dist = get_sqrt_dist(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Root sum of squares {sqrt_dist:3f}")
    print()


In [11]:
report_ideal(ideal_distrib)

Ideal input states
POVM for ideal optimal UQSD
[0.3205 0.2667 0.2237 0.1891]



In [12]:
for i in range(len(noise_levels)):
    report_noisy_states_with_ideal_povm(noise_levels[i], ideal_distrib)

Noisy input states with depolarizing noise = 1e-06
POVM for ideal optimal UQSD
[0.3205 0.2667 0.2237 0.1891]
alpha [4.1191e-07 5.3197e-07 6.8428e-07]
beta  [5.0831e-07 5.3722e-07 5.3992e-07]
Jensen-Shannon dist 0.000000
Root sum of squares 0.000000

Noisy input states with depolarizing noise = 3.16228e-06
POVM for ideal optimal UQSD
[0.3205 0.2667 0.2237 0.1891]
alpha [1.3026e-06 1.6822e-06 2.1639e-06]
beta  [1.6074e-06 1.6988e-06 1.7074e-06]
Jensen-Shannon dist 0.000000
Root sum of squares 0.000001

Noisy input states with depolarizing noise = 1e-05
POVM for ideal optimal UQSD
[0.3205 0.2667 0.2237 0.1891]
alpha [4.1191e-06 5.3197e-06 6.8428e-06]
beta  [5.0832e-06 5.3722e-06 5.3992e-06]
Jensen-Shannon dist 0.000002
Root sum of squares 0.000002

Noisy input states with depolarizing noise = 3.16228e-05
POVM for ideal optimal UQSD
[0.3205 0.2667 0.2237 0.1891]
alpha [1.3026e-05 1.6822e-05 2.1639e-05]
beta  [1.6074e-05 1.6988e-05 1.7074e-05]
Jensen-Shannon dist 0.000005
Root sum of square

In [13]:
for i in range(len(noise_levels)):
    report_noisy_states_with_FRIO_povm(noise_levels[i], ideal_distrib)

Noisy input states with depolarizing noise = 1e-06
POVM for FRIO-optimal UQSD
[0.3198 0.2645 0.2266 0.1891]
alpha [1.7293e-06 1.6575e-06 1.9451e-06]
beta  [1.3578e-06 1.7662e-06 2.3425e-06]
Jensen-Shannon dist 0.002606
Root sum of squares 0.003615

Noisy input states with depolarizing noise = 3.16228e-06
POVM for FRIO-optimal UQSD
[0.3198 0.2645 0.2266 0.1891]
alpha [5.5122e-07 3.8684e-07 3.7959e-06]
beta  [1.5517e-06 1.7646e-06 7.7501e-07]
Jensen-Shannon dist 0.002605
Root sum of squares 0.003614

Noisy input states with depolarizing noise = 1e-05
POVM for FRIO-optimal UQSD
[0.3198 0.2646 0.2266 0.1891]
alpha [3.1817e-06 4.6127e-06 9.6459e-06]
beta  [5.9182e-06 5.9782e-06 4.1888e-06]
Jensen-Shannon dist 0.002601
Root sum of squares 0.003608

Noisy input states with depolarizing noise = 3.16228e-05
POVM for FRIO-optimal UQSD
[0.3198 0.2646 0.2265 0.1891]
alpha [1.5061e-05 1.9698e-05 2.8121e-05]
beta  [1.9722e-05 2.1055e-05 1.9955e-05]
Jensen-Shannon dist 0.002558
Root sum of squares 0.

In [14]:
# Solve with epsilon-UQSD
def report_noisy_states_with_epsilon_povm(noise_level, ideal_distrib):
    combined_states = [
        (1 - noise_level) * dense_mat[_]
        + noise_level * disturbance_states[_].data
        for _ in range(num_states)
    ]
    
    problem_asymm_states_1.set_states(
        state_type="densitymatrix",
        states=combined_states,
        overwrite=True,
    )

    alpha = [noise_level * 0.75] * problem_asymm_states_1.num_states
    print(alpha)
    beta = [noise_level * 0.75] * problem_asymm_states_1.num_states
    print(beta)

    noisy_result = apply_crossQD(
        problem_spec=problem_asymm_states_1,
        alpha=alpha,
        beta=beta,
        is_cvxpy_verbose=False,
        max_iters=int(1e5),
        eps_abs=1e-7,
        eps_rel=1e-7,
    )

    print(len(noisy_result["PI_list"]))

    # m is prob_matrix
    m = calculate_prob_matrix_simple(
        prior_probs=problem_asymm_states_1.prior_prob,
        povm=noisy_result["PI_list"],
        states=combined_states,
        ## bitstring_to_target_state=result["bitstring_to_target_state"],
        ## strings_used=result["strings_used"],
    )
    for i in m:
        print(i)
    noisy_distrib_with_ideal_povm = dist_2(m)
    print(f"Noisy input states with depolarizing noise = {noise_level:3g}")
    print("POVM for epsilon UQSD")
    print(noisy_distrib_with_ideal_povm)
    np.set_printoptions(precision=4)
    print("alpha", calculate_errors(m)[0])
    # print("alpha", np.array(calculate_errors(m)[0]))
    print("beta ", calculate_errors(m)[1])
    # print("beta ", np.array(calculate_errors(m)[1]))
    js_dist = jensenshannon(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Jensen-Shannon dist {js_dist:3f}")
    sqrt_dist = get_sqrt_dist(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Root sum of squares {sqrt_dist:3f}")
    print()


In [15]:
for i in range(len(noise_levels)):
    report_noisy_states_with_epsilon_povm(noise_levels[i], ideal_distrib)

[7.500000000000002e-07, 7.500000000000002e-07, 7.500000000000002e-07]
[7.500000000000002e-07, 7.500000000000002e-07, 7.500000000000002e-07]
Result = 0.8116573876997932
CVXPY returns optimal
Probabilities for each state:
[np.float64(0.32048866232648665), np.float64(8.618626140545958e-08), np.float64(8.754291191533816e-08), np.float64(0.012844496434114117)]
[np.float64(9.8101148707508e-08), np.float64(0.26617179557081494), np.float64(9.332701990296538e-08), np.float64(0.06716135331567681)]
[np.float64(1.1769720528598945e-07), np.float64(1.1029947929011842e-07), np.float64(0.22499693156985445), np.float64(0.10833619574298821)]
Total probability = 1.0000
Success probability = 0.8117
Inconclusive probability = 0.1883

4
[0.3204886622077008, 9.632061518688048e-08, 9.142228791835228e-08, 0.012844483435137277]
[1.0301418803948237e-07, 0.2661717944548059, 8.329185539420099e-08, 0.06716135261059093]
[1.1158654288344083e-07, 9.411923858336086e-08, 0.22499693103728663, 0.10833619659143864]
Noisy i

/home/ChienKaiMa/QSD/.venv/lib/python3.12/site-packages/diffcp/cone_program.py:348: UserWarning: Solved/Inaccurate.
  warnings.warn("Solved/Inaccurate.")
/home/ChienKaiMa/QSD/.venv/lib/python3.12/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Result = 0.8122371367800683
CVXPY returns optimal_inaccurate
Probabilities for each state:
[np.float64(0.3204713864058767), np.float64(3.2700489689034775e-07), np.float64(2.7900282115990416e-07), np.float64(0.01286131048502832)]
[np.float64(3.4301866380431827e-07), np.float64(0.26579863292837375), np.float64(2.9840670196837945e-07), np.float64(0.06753408443817588)]
[np.float64(3.7049270178743855e-07), np.float64(4.0029254507368976e-07), np.float64(0.22596712508709133), np.float64(0.10736556247100676)]
Total probability = 1.0000
Success probability = 0.8122
Inconclusive probability = 0.1878

4
[0.3204713861892602, 3.267031289510532e-07, 3.099540788212231e-07, 0.012861310486858974]
[3.428144739529748e-07, 0.2657986263525252, 2.797271058772363e-07, 0.06753408443922419]
[3.4961878504291785e-07, 2.970043544091589e-07, 0.22596712423828283, 0.10736556247190888]
Noisy input states with depolarizing noise = 3.16228e-06
POVM for epsilon UQSD
[0.3205 0.2658 0.226  0.1878]
alpha [np.float64(1.9866

In [16]:
def report_euqsd_povm(noise_level):
    combined_states = [
        (1 - noise_level) * dense_mat[_]
        + noise_level * disturbance_states[_].data
        for _ in range(num_states)
    ]

    problem_asymm_states_1.set_states(
        state_type="densitymatrix",
        states=combined_states,
        overwrite=True,
    )

    alpha = [noise_level * 0.75] * problem_asymm_states_1.num_states
    print(alpha)
    beta = [noise_level * 0.75] * problem_asymm_states_1.num_states
    print(beta)

    noisy_result = apply_crossQD(
        problem_spec=problem_asymm_states_1,
        alpha=alpha,
        beta=beta,
        is_cvxpy_verbose=False,
        max_iters=int(1e6),
        eps_abs=1e-7,
        eps_rel=1e-10,
    )

    return noisy_result["PI_list"]

def report_noisy_states_with_povm(povm, noise_level, ideal_distrib):
    combined_states = [
        (1 - noise_level) * dense_mat[_]
        + noise_level * disturbance_states[_].data
        for _ in range(num_states)
    ]
    
    problem_asymm_states_1.set_states(
        state_type="densitymatrix",
        states=combined_states,
        overwrite=True,
    )
    # m is prob_matrix
    m = calculate_prob_matrix_simple(problem_asymm_states_1.prior_prob, povm, combined_states)
    # for i in m:
    #     print(i)
    noisy_distrib_with_ideal_povm = dist_2(m)
    print(f"Noisy input states with depolarizing noise = {noise_level:3g}")
    print("POVM for epsilon UQSD")
    print(noisy_distrib_with_ideal_povm)
    np.set_printoptions(precision=4)
    print("alpha", np.array(calculate_errors(m)[0]))
    print("beta ", np.array(calculate_errors(m)[1]))
    js_dist = jensenshannon(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Jensen-Shannon dist {js_dist:3f}")
    sqrt_dist = get_sqrt_dist(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Root sum of squares {sqrt_dist:3f}")
    print()


In [17]:
povm = report_euqsd_povm(noise_level=0.01)
for i in range(len(noise_levels)):
    report_noisy_states_with_povm(povm, noise_levels[i], ideal_distrib)

[0.0075, 0.0075, 0.0075]
[0.0075, 0.0075, 0.0075]


Result = 0.8546556096970978
CVXPY returns optimal
Probabilities for each state:
[np.float64(0.3194285654437823), np.float64(0.0007136699397432445), np.float64(0.0008433625802092006), np.float64(0.012347748358590375)]
[np.float64(0.0008158397637638848), np.float64(0.26579271293104756), np.float64(0.0011927241373763382), np.float64(0.0655320876767291)]
[np.float64(0.0008958376961510012), np.float64(0.0011401433276027636), np.float64(0.2694343319001511), np.float64(0.061862997958022754)]
Total probability = 1.0000
Success probability = 0.8547
Inconclusive probability = 0.1397

Noisy input states with depolarizing noise = 1e-06
POVM for epsilon UQSD
[0.3219 0.2682 0.272  0.1378]
alpha [0.0004 0.0018 0.0019]
beta  [0.0003 0.0017 0.0022]
Jensen-Shannon dist 0.056566
Root sum of squares 0.070485

Noisy input states with depolarizing noise = 3.16228e-06
POVM for epsilon UQSD
[0.3219 0.2682 0.272  0.1378]
alpha [0.0004 0.0018 0.0019]
beta  [0.0003 0.0017 0.0022]
Jensen-Shannon dist 0.056565
Roo

In [18]:
povm = report_euqsd_povm(noise_level=0.001)
for i in range(len(noise_levels)):
    report_noisy_states_with_povm(povm, noise_levels[i], ideal_distrib)

[0.00075, 0.00075, 0.00075]
[0.00075, 0.00075, 0.00075]
Result = 0.8260665550378585
CVXPY returns optimal
Probabilities for each state:
[np.float64(0.3189566364845516), np.float64(9.457487168719494e-05), np.float64(7.785265265659289e-05), np.float64(0.01420428135596883)]
[np.float64(8.353553471829588e-05), np.float64(0.2574068478887947), np.float64(0.00010961182717237881), np.float64(0.07573331824598317)]
[np.float64(8.877949343729726e-05), np.float64(9.860924633378118e-05), np.float64(0.24970306993884897), np.float64(0.08344285851870567)]
Total probability = 1.0000
Success probability = 0.8261
Inconclusive probability = 0.1734

Noisy input states with depolarizing noise = 1e-06
POVM for epsilon UQSD
[0.3192 0.2577 0.2499 0.1732]
alpha [0.0001 0.0002 0.0002]
beta  [3.2427e-05 2.1560e-04 2.1499e-04]
Jensen-Shannon dist 0.024018
Root sum of squares 0.031986

Noisy input states with depolarizing noise = 3.16228e-06
POVM for epsilon UQSD
[0.3192 0.2577 0.2499 0.1732]
alpha [0.0001 0.0002 0

In [19]:
def report_noisy_states_with_crossqd2_povm(noise_level, ideal_distrib):
    combined_states = [
        (1 - noise_level) * dense_mat[_]
        + noise_level * disturbance_states[_].data
        for _ in range(num_states)
    ]
    
    problem_asymm_states_1.set_states(
        state_type="densitymatrix",
        states=combined_states,
        overwrite=True,
    )
    # TODO for the noise level, calculate_error for ideal povm
    m = calculate_prob_matrix_simple(problem_asymm_states_1.prior_prob, ideal_result["povm"], combined_states)
    alpha = np.array(calculate_errors(m)[0])
    beta = np.array(calculate_errors(m)[1])

    noisy_result = apply_crossQD_2(
        problem_spec=problem_asymm_states_1,
        alpha=alpha,
        beta=beta,
        gamma=ideal_distrib[-1],
        is_cvxpy_verbose=False,
    )
    # m is prob_matrix
    m = calculate_prob_matrix_simple(
        prior_probs=problem_asymm_states_1.prior_prob,
        povm=noisy_result["PI_list"],
        states=combined_states,
        ## bitstring_to_target_state=result["bitstring_to_target_state"],
        ## strings_used=result["strings_used"],
    )
    # for i in m:
    #     print(i)
    noisy_distrib_with_ideal_povm = dist_2(m)
    print(f"Noisy input states with depolarizing noise = {noise_level:3g}")
    print("POVM for e-UQSD")
    print(noisy_distrib_with_ideal_povm)
    np.set_printoptions(precision=4)
    print("alpha", np.array(calculate_errors(m)[0]))
    print("beta ", np.array(calculate_errors(m)[1]))
    js_dist = jensenshannon(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Jensen-Shannon dist {js_dist:3f}")
    sqrt_dist = get_sqrt_dist(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Root sum of squares {sqrt_dist:3f}")
    print()


In [23]:
for i in range(len(noise_levels)):
    report_noisy_states_with_crossqd2_povm(noise_levels[i], ideal_distrib)

Noisy input states with depolarizing noise = 1e-06
POVM for e-UQSD
[0.3205 0.2667 0.2237 0.1891]
alpha [4.2520e-07 5.4923e-07 7.0297e-07]
beta  [5.2129e-07 5.5367e-07 5.6001e-07]
Jensen-Shannon dist 0.000003
Root sum of squares 0.000004

Noisy input states with depolarizing noise = 3.16228e-06
POVM for e-UQSD
[0.3205 0.2667 0.2237 0.1891]
alpha [1.3003e-06 1.6797e-06 2.1618e-06]
beta  [1.6057e-06 1.6965e-06 1.7043e-06]
Jensen-Shannon dist 0.000003
Root sum of squares 0.000004

Noisy input states with depolarizing noise = 1e-05
POVM for e-UQSD
[0.3205 0.2667 0.2237 0.1891]
alpha [4.1156e-06 5.3139e-06 6.8360e-06]
beta  [5.0788e-06 5.3670e-06 5.3929e-06]
Jensen-Shannon dist 0.000006
Root sum of squares 0.000008

Noisy input states with depolarizing noise = 3.16228e-05
POVM for e-UQSD
[0.3205 0.2666 0.2238 0.1891]
alpha [1.3015e-05 1.6807e-05 2.1614e-05]
beta  [1.6061e-05 1.6973e-05 1.7054e-05]
Jensen-Shannon dist 0.000054
Root sum of squares 0.000075

Noisy input states with depolarizing

In [21]:
def report_noisy_states_with_crossqd3_povm(noise_level, ideal_distrib):
    combined_states = [
        (1 - noise_level) * dense_mat[_]
        + noise_level * disturbance_states[_].data
        for _ in range(num_states)
    ]
    
    problem_asymm_states_1.set_states(
        state_type="densitymatrix",
        states=combined_states,
        overwrite=True,
    )
    # TODO for the noise level, calculate_error for ideal povm
    m = calculate_prob_matrix_simple(problem_asymm_states_1.prior_prob, ideal_result["povm"], combined_states)
    alpha = np.array(calculate_errors(m)[0])
    beta = np.array(calculate_errors(m)[1])

    noisy_result = apply_crossQD_3(
        ideal_distrib=ideal_distrib,
        problem_spec=problem_asymm_states_1,
        gamma=ideal_distrib[-1],
        is_cvxpy_verbose=False,
    )
    # m is prob_matrix
    m = calculate_prob_matrix_simple(
        prior_probs=problem_asymm_states_1.prior_prob,
        povm=noisy_result["PI_list"],
        states=combined_states,
        ## bitstring_to_target_state=result["bitstring_to_target_state"],
        ## strings_used=result["strings_used"],
    )
    # for i in m:
    #     print(i)
    noisy_distrib_with_ideal_povm = dist_2(m)
    print(f"Noisy input states with depolarizing noise = {noise_level:3g}")
    print("POVM for crossQD3")
    print(noisy_distrib_with_ideal_povm)
    np.set_printoptions(precision=4)
    print("alpha", np.array(calculate_errors(m)[0]))
    print("beta ", np.array(calculate_errors(m)[1]))
    js_dist = jensenshannon(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Jensen-Shannon dist {js_dist:3f}")
    sqrt_dist = get_sqrt_dist(ideal_distrib, noisy_distrib_with_ideal_povm)
    print(f"Root sum of squares {sqrt_dist:3f}")
    print()


In [22]:
for i in range(len(noise_levels)):
    report_noisy_states_with_crossqd3_povm(noise_levels[i], ideal_distrib)

Noisy input states with depolarizing noise = 1e-06
POVM for crossQD3
[0.3205 0.2667 0.2237 0.1891]
alpha [6.9605e-07 6.4670e-07 7.0938e-07]
beta  [4.7899e-07 7.3458e-07 9.1560e-07]
Jensen-Shannon dist 0.000000
Root sum of squares 0.000000

Noisy input states with depolarizing noise = 3.16228e-06
POVM for crossQD3
[0.3205 0.2667 0.2237 0.1891]
alpha [1.1897e-06 1.7531e-06 2.1333e-06]
beta  [1.6517e-06 1.5569e-06 1.7053e-06]
Jensen-Shannon dist 0.000000
Root sum of squares 0.000000

Noisy input states with depolarizing noise = 1e-05
POVM for crossQD3
[0.3205 0.2667 0.2237 0.1891]
alpha [3.7939e-06 5.5607e-06 6.9173e-06]
beta  [5.3691e-06 5.0442e-06 5.2763e-06]
Jensen-Shannon dist 0.000000
Root sum of squares 0.000000

Noisy input states with depolarizing noise = 3.16228e-05
POVM for crossQD3
[0.3205 0.2667 0.2237 0.1891]
alpha [1.2733e-05 1.6987e-05 2.1838e-05]
beta  [1.6369e-05 1.6802e-05 1.6851e-05]
Jensen-Shannon dist 0.000000
Root sum of squares 0.000000

Noisy input states with depo